In [1]:
import pandas as pd
from datetime import datetime, timedelta
from gamma import BayesianGaussianMixture, GaussianMixture
from gamma.utils import convert_picks_csv, association, from_seconds, initialize_eikonal
import numpy as np
from sklearn.cluster import DBSCAN 
from datetime import datetime, timedelta
import os
import json
import pickle
from pyproj import Proj
from tqdm import tqdm
import time
import glob
import matplotlib.pyplot as plt

In [2]:
picks = pd.read_csv('./phases_new_run/2345_picks_250320.csv')
picks.phase_index = np.arange(len(picks))
picks.index = np.arange(len(picks))
picks['phase_time'] = pd.to_datetime(picks['phase_time']).dt.tz_convert('utc')
picks.sort_values(by='phase_time', inplace=True)

selected = picks.station=='V0106'
picks.loc[selected, 'station'] = 'CLAC'
picks.loc[selected, 'station_id'] = '2I.CLAC..HH'
picks.loc[picks.station=='CAWE', 'station_id'] = 'IV.CAWE..EH'

In [3]:
picks["id"] = picks["station_id"]
picks["timestamp"] = picks["phase_time"]
# picks["amp"] = picks['abs_max']
picks["type"] = picks["phase_type"]
picks["prob"] = picks["phase_score"]
picks_count_bass = picks.station_id.value_counts()
picks_count_bass

In [4]:
config = {'center': (14.13, 40.81),  
        'xlim_degree': [14.0, 14.2], 
        'ylim_degree': [40.7, 40.9], 
        'degree2km': 111.19492474777779, 
        'starttime': datetime(2022, 1, 11, 0, 0), 
        'endtime':   datetime(2025, 3, 20, 0, 0)}
stations = pd.read_json('./stations.json')
stations = stations.transpose()
stations['id'] = stations.index
proj = Proj(f"+proj=sterea +lon_0={config['center'][0]} +lat_0={config['center'][1]} +units=km")
stations[["x(km)", "y(km)"]] = stations.apply(lambda x: pd.Series(proj(longitude=x.longitude, latitude=x.latitude)), axis=1)
stations["z(km)"] = stations["elevation(m)"].apply(lambda x: -x/1e3)
stations['component'] = [['E', 'N', 'Z']] * len(stations)
stations

In [5]:
### setting GMMA configs
config["use_dbscan"] = True
config["use_amplitude"] = False
config["method"] = "BGMM"  
if config["method"] == "BGMM": ## BayesianGaussianMixture
    config["oversample_factor"] = 5
if config["method"] == "GMM": ## GaussianMixture
    config["oversample_factor"] = 1

config["dims"] = ['x(km)', 'y(km)', 'z(km)']
config["x(km)"] = (np.array(config["xlim_degree"])-np.array(config["center"][0]))*config["degree2km"]*np.cos(np.deg2rad(config["center"][1]))
config["y(km)"] = (np.array(config["ylim_degree"])-np.array(config["center"][1]))*config["degree2km"]
config["z(km)"] = (0, 20)
config["bfgs_bounds"] = (
    (config["x(km)"][0] - 1, config["x(km)"][1] + 1),  # x
    (config["y(km)"][0] - 1, config["y(km)"][1] + 1),  # y
    (config["z(km)"][0], config["z(km)"][1] + 1),  # z
    (None, None),  # t
)

# DBSCAN
config["dbscan_eps"] = 5 #s
config["dbscan_min_samples"] = 3

zz = [   0,  0.5,  1.0,  1.5,  2.0,  3.0]
vp = [1.81, 2.33, 2.71, 3.76, 3.89, 4.51]
vs = [1.02, 1.02, 1.46, 2.21, 2.49, 2.96]

# vp_vs_ratio = 1.73
h = 0.2
vel = {"z": zz, "p": vp, "s": vs}
config["eikonal"] = {"vel": vel, "h": h, "xlim": config["x(km)"], "ylim": config["y(km)"], "zlim": config["z(km)"]}

# filtering
config["min_picks_per_eq"] = 6
config["min_p_picks_per_eq"] = 0
config["min_s_picks_per_eq"] = 0
config["max_sigma11"] = 1.0  # s
config["max_sigma22"] = 1.0  # log10(m/s)
config["max_sigma12"] = 1.0  # covariance

## filter picks without amplitude measurements
if config["use_amplitude"]:
    picks = picks[picks["phase_score"] != -1]

for k, v in config.items():
    print(f"{k}: {v}")

In [6]:
begin_time = time.time()
event_idx0 = 0 ## current earthquake index
assignments = []
catalogs, assignments = association(picks, stations, config, event_idx0, config["method"])
event_idx0 += len(catalogs)
end_time = time.time()
print(f"Total time: {end_time-begin_time:.2f} s")

In [9]:
## create catalog
catalogs = pd.DataFrame(catalogs, columns=["time"]+config["dims"]+["magnitude", "sigma_time", "sigma_amp", "cov_time_amp",  "event_index", "gamma_score"])
catalogs[["longitude","latitude"]] = catalogs.apply(lambda x: pd.Series(proj(longitude=x["x(km)"], latitude=x["y(km)"], inverse=True)), axis=1)
catalogs["depth(m)"] = catalogs["z(km)"].apply(lambda x: x*1e3)
catalogs["magnitude"] = [0]*len(catalogs)
catalogs.sort_values('time')

In [11]:
catalog_pth = './gamma_catalog_0320.csv'
pick_pth = './gamma_pick_0320.csv'
with open(catalog_pth, 'w') as fp:
    catalogs.to_csv(fp, index=False, 
                    float_format="%.3f",
                    date_format='%Y-%m-%dT%H:%M:%S.%f',
                    columns=["time", "magnitude", "longitude", "latitude", "depth(m)", "sigma_time", "sigma_amp", "cov_time_amp", "event_index", "gamma_score"])
# add assignment to picks
assignments = pd.DataFrame(assignments, columns=["pick_index", "event_index", "gamma_score"])
pick = picks.join(assignments.set_index("pick_index")).fillna(-1).astype({'event_index': int})
pick = pick[pick.event_index!=-1]
with open(pick_pth, 'w') as fp:
    pick.to_csv(fp, index=False, 
                date_format='%Y-%m-%dT%H:%M:%S.%f',
                columns=["station_id", "phase_time", "phase_type", "phase_score", "event_index", "gamma_score"])